## RLDMUU 2025
#### Backward Induction
jakub.tluczek@unine.ch

Today your task would be to implement the backwards induction algorithm for the following MDP (you can also find it in `src/MDP/MDP.py` on our github):

In [1]:
import numpy as np

## This a discrete MDP with a finite number of states and actions
class DiscreteMDP:
    ## initalise a random MDP with
    ## n_states: the number of states
    ## n_actions: the number of actions
    ## Optional arguments:
    ## P: the state-action-state transition matrix so that P[s,a,s_next] is the probability of s_next given the current state-action pair (s,a)
    ## R: The state-action reward matrix so that R[s,a] is the reward for taking action a in state s.
    def __init__(self, n_states, n_actions, P = None, R = None):
        self.n_states = n_states # the number of states of the MDP
        self.n_actions = n_actions # the number of actions of the MDP
        if (P is None):
            self.P = np.zeros([n_states, n_actions, n_states]) # the transition probability matrix of the MDP so that P[s,a,s'] is the probabiltiy of going to s' from (s,a)
            for s in range(self.n_states):
                for a in range(self.n_actions):
                    self.P[s,a] = np.random.dirichlet(np.ones(n_states)) # generalisation of Beta to multiple outcome
        else:
            self.P = P
        if (R is None):
            self.R = np.zeros([n_states, n_actions]) # the expected reward for each action and state
            # generate uniformly random transitions and 0.1 bernoulli rewards
            for s in range(self.n_states):
                for a in range(self.n_actions):
                    self.R[s,a] = np.round(np.random.uniform(), decimals=1)
        else:
            self.R = R

        # check transitions
        for s in range(self.n_states):
            for a in range(self.n_actions):
                #print(s,a, ":", self.P[s,a,:])
                assert(abs(np.sum(self.P[s,a,:])-1) <= 1e-3)
                assert((self.P[s,a,:] <= 1).all())
                assert((self.P[s,a,:] >= 0).all())

    # get the probability of next state j given current state s, action a, i.e. P(j|s,a)
    def get_transition_probability(self, state, action, next_state):
        return self.P[state, action, next_state]

    # get the vector of probabilities over next states P( . | s,a)
    def get_transition_probabilities(self, state, action):
        return self.P[state, action]

    # Get the reward for the current state action.
    # It can also be interpreted as the expected reward for the state and action.
    def get_reward(self, state, action):
        return self.R[state, action]

### Backward induction

As a reminder, in the backward induction algorithm we consider an MDP with finite horizon $T$, and for each step of the algorithm, we compute:

$$ V_t(s) = max_{a \in A} \left[ R(s,a) + \sum_{s' \in S} P(s' | s,a)V_{t+1}(s') \right]$$

where $R(s,a)$ is a reward received by picking action $a$ in state $s$, $P(s'|s,a)$ is the probability of transitioning into next state $s'$, and $V_{t+1}(s')$ is the value of said next state at time $t+1$. We can also say, that for the last timestep (with index $T-1$) the next state value is 0 for every state $V_T(s) = 0$. Consecutively, the action $a$ which maximizes $V_t(s)$ can be described as policy $\pi_t(s)$ at state $s$ and time $t$,

Your task is to implement this algorithm. Remember to do the inverse iteration, and iterate from $T-1$ to $0$, not the other way around. Your function should return matrix of state values for each $s$ and $t$, as well as resulting policy:

In [ ]:
# TODO: Implement backwards induction
def backwards_induction(mdp, T):
    V = np.zeros([mdp.n_states, T])
    policy = np.zeros([mdp.n_states, T])
    for t in range(T-1, -1, -1):
        for s in range(mdp.n_states):
            max_value = -np.inf
            for a in range(mdp.n_actions):
                value = mdp.get_reward(s, a)
                for s_next in range(mdp.n_states):
                    # Check if t+1 is within bounds before accessing V
                    if t + 1 < T:
                        value += mdp.get_transition_probability(s, a, s_next) * V[s_next, t+1]
                if value > max_value:
                    max_value = value
                    policy[s, t] = a
            V[s, t] = max_value
    return V, policy

In [ ]:
STATES = 5
ACTIONS = 3

T = 15

mdp = DiscreteMDP(STATES, ACTIONS)

V, policy = backwards_induction(mdp, T)

print(V)

print(policy)


[[11.3398275  10.60101919  9.86221093  9.12340354  8.38459643  7.64578368
   6.90695493  6.16817261  5.42954119  4.69088384  3.95025256  3.20898284
   2.47862885  1.78817336  1.        ]
 [11.02017955 10.28137138  9.54256276  8.80375357  8.06494757  7.32615132
   6.58733932  5.84841954  5.10948717  4.37152142  3.63495566  2.89180343
   2.12641368  1.38933365  0.8       ]
 [11.49707933 10.75827111 10.01946323  9.28065537  8.54184503  7.80302947
   7.06423393  6.32549703  5.58671238  4.84717222  4.10753511  3.37289306
   2.65132597  1.88614047  1.        ]
 [10.57822696  9.83941884  9.10061072  8.36180178  7.62299142  6.88418746
   6.14540218  5.40659441  4.66755838  3.92850227  3.19116998  2.45735747
   1.70989459  0.92224429  0.2       ]
 [11.23903579 10.50022752  9.76141914  9.02261101  8.28380485  7.54499726
   6.8061759   6.06732946  5.32863245  4.59017464  3.85132058  3.1078059
   2.36766648  1.6538563   1.        ]]
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [2. 2. 2. 2. 2. 